In [4]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

In [31]:
# TODO: get_distance_matrix 
"""
params : 
 feature_vector: array of feature vectors for each shot 
return : 2D distance matrix (Matrix 2D with size N*N ,N is the numer of shots,cell (i,j) have the feature distance between i,j shots)
"""
features_matrix =  np.array([[2, 1, 0, 4, 1], [0, 3, 1, 1, 10],[1, 1, 0, 1, 0]])
def get_distance_matrix(features_matrix):
    D_Matrix = sparse.csr_matrix(features_matrix)
    D_Matrix = cosine_similarity(D_Matrix)
    D_Matrix = 1-D_Matrix
    # D_Matrix.tolist()
    return D_Matrix

#TODO: try eculedian  distance 

In [32]:
D_Matrix = get_distance_matrix(features_matrix)
print(D_Matrix)
print(1-D_Matrix)

[[ 2.22044605e-16  6.55986020e-01  1.38359563e-01]
 [ 6.55986020e-01 -2.22044605e-16  7.80801350e-01]
 [ 1.38359563e-01  7.80801350e-01 -2.22044605e-16]]
[[1.         0.34401398 0.86164044]
 [0.34401398 1.         0.21919865]
 [0.86164044 0.21919865 1.        ]]


In [37]:
#TODO: get_internal_sums
"""get the total sum of distance of different segmentation 
params :
 Distance_Matrix: 2D distance matrix 
 N: number of shots 
 start_shot: begin of scene 
 end_shot: end of scene 
return : table with all different segments in D_Matrix
"""
def get_internal_sums(D_Matrix,N):
    # D_sum = torch.zeros(N,N, device=self.device)
    D_sum = np.zeros(shape=(N,N))
    # initialize diagonal 
    for shot_index in range(N):
        D_sum[shot_index,shot_index] = D_Matrix[shot_index,shot_index]
    # initialize second diagonal
    for shot_index in range(0, N-1):
        D_sum[shot_index, shot_index+1] = D_Matrix[shot_index:shot_index+1+1, shot_index:shot_index+1+1].sum()
        print(D_sum[shot_index, shot_index+1])
        # TODO: recheck this condition 
        D_sum[shot_index+1, shot_index] = D_Matrix[shot_index, shot_index+1]
        print(D_sum[shot_index+1, shot_index])
    for start_shot in range(2, N):
        for end_shot in range(0, N - start_shot):
            # D_sum[i,j] = +D_sum[i-1,j]                  --> missing last row
            #              +D_sum[i,j-1]                  --> missing last column 
            #              -D_sum[i-1,j-1]                --> as we sum it twice before
            #              +D_Matrix[i,j] + D_Matrix[j,i] -->missing cells in brevious calculations  
            D_sum[end_shot, end_shot + start_shot] = D_sum[end_shot, end_shot + start_shot - 1] + D_sum[end_shot - 1, end_shot + start_shot] - D_sum[end_shot - 1, end_shot + start_shot - 1] +  D_Matrix[end_shot, end_shot + start_shot] + D_Matrix[end_shot + start_shot, end_shot] 
            # as the matrix is symetric 
            D_sum[end_shot + start_shot, end_shot] = D_sum[end_shot, end_shot + start_shot]
    return D_sum

In [38]:
D_sum = get_internal_sums(D_Matrix,3)
print(D_sum)

1.3119720402611001
0.6559860201305501
1.5616027005190467
0.7808013502595236
[[ 2.22044605e-16  1.31197204e+00  8.07889816e-01]
 [ 6.55986020e-01 -2.22044605e-16  1.56160270e+00]
 [ 8.07889816e-01  7.80801350e-01 -2.22044605e-16]]


In [39]:
# TODO: hierarchical clustering_h_add

"""
params : 2D distance matrix (with size N*N)
return : T (T is the optimal shot indexes where thw shots are devided into scenes)

"""

'\nparams : 2D distance matrix (with size N*N)\nreturn : T (T is the optimal shot indexes where thw shots are devided into scenes)\n\n'

In [ ]:
# TODO: hierarchical clustering_h_nrm

"""
params : 2D distance matrix (with size N*N)
return : T (T is the optimal shot indexes where thw shots are devided into scenes)
"""
